In [1]:
%pip install lightning

import numpy
import pandas
import matplotlib.pyplot as plt
import csv
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import sklearn
from math import sqrt
import lightning as pl


Note: you may need to restart the kernel to use updated packages.


https://www.pluralsight.com/guides/predictive-analytics-with-pytorch

In [2]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/ceara_school_count_data.csv")

schools = schools.iloc[:, 3:]

schools = schools.dropna()

df = schools

df.head()

target_column = ['NUMPOINTS'] 
predictors = list(set(list(df.columns))-set(target_column))

print(target_column)
print(predictors)

X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 30)
print(X_train.shape); print(X_test.shape)


['NUMPOINTS']
['smod', 'x', 'y', 'ID', 'built_v', 'built_s', 'land', 'pop']
(105854, 8)
(45367, 8)


In [3]:
class ZIPModel(pl.LightningModule):
    def __init__(
        self,
        learning_rate=1e-3,
        init_mix_parameter: float = 0.5,
        init_poisson_lambda: float = 1.0,
    ):
        super().__init__()
        self.mixture_prob = torch.nn.Parameter(torch.tensor([init_mix_parameter]))
        self.poisson_lambda = torch.nn.Parameter(torch.tensor([init_poisson_lambda]))

        self.train_log_error = []
        self.val_log_error = []
        self.mixture_prob_log = []
        self.poisson_lambda_log = []
        self.learning_rate = learning_rate

    def forward(self):
        # ensure correct domain for params
        mixture_prob_norm = torch.sigmoid(self.mixture_prob)
        poisson_lambda_norm = torch.nn.functional.softplus(self.poisson_lambda)
        poisson_lambda_norm = torch.concat(
            (torch.tensor([0.0]), poisson_lambda_norm)
        )  # maintains grad

        mix = torch.distributions.Categorical(
            torch.concat((mixture_prob_norm, 1 - mixture_prob_norm))
        )
        poissons = torch.distributions.Poisson(poisson_lambda_norm)

        mixture_dist = torch.distributions.MixtureSameFamily(mix, poissons)
        return mixture_dist

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.learning_rate,
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        y = batch[0]

        mixture_dist = self.forward()

        negloglik = -mixture_dist.log_prob(y)
        loss = torch.mean(negloglik)

        self.train_log_error.append(loss.detach().numpy())
        self.poisson_lambda_log.append(
            torch.nn.functional.softplus(self.poisson_lambda).detach().numpy()
        )
        self.mixture_prob_log.append(torch.sigmoid(self.mixture_prob).detach().numpy())
        return loss

    def validation_step(self, batch, batch_idx):
        y = batch[0]

        mixture_dist = self.forward()

        negloglik = -mixture_dist.log_prob(y)
        loss = torch.mean(negloglik)

        self.train_log_error.append(loss.detach().numpy())
        return loss


In [4]:
model = ZIPModel(learning_rate=1e-0, init_mix_parameter=0.5, init_poisson_lambda=4.5)

print(model)

ZIPModel()


In [5]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)#.view(-1,1)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)#.view(-1,1)

train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 64, shuffle = True)

In [2]:
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

trainer = pl.Trainer(
    max_epochs=100,
)
trainer.fit(model, train_loader)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/lightning_fabric/accelerators/cuda.py:241: Can't initialize NVML
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NameError: name 'model' is not defined